In [42]:
import numpy as np
import pandas as pd

In [43]:
df = pd.read_csv('gurgaon_properties_post_feature_selection_updated.csv')

In [44]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,11.0,3,2,2.0,3.0,717.0,0,0,0,1.0,2.0,0.50
1,0.0,104.0,2,2,1.0,1.0,767.0,0,0,0,1.0,1.0,0.47
2,0.0,96.0,3,3,2.0,3.0,1778.0,1,0,1,1.0,2.0,1.19
3,0.0,75.0,2,2,2.0,3.0,1188.0,0,0,1,2.0,2.0,1.51
4,0.0,71.0,2,2,3.0,4.0,1676.0,0,0,0,2.0,0.0,2.15


In [30]:
# one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category

In [45]:
X = df.drop(columns=['price'])
y = df['price']

In [46]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [47]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [48]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [49]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [50]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [51]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

C:\Users\abhin\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py:971: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\abhin\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_scorer.py", line 152, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\abhin\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_scorer.py", line 400, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "C:\Users\abhin\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_scorer.py", line 90, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abhin\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\_response.py", line 242, in _get_response_values
    y_pred, pos_label = prediction_method(X), None
      

In [38]:
scores.mean()

0.8857537050981398

In [39]:
scores.std()

0.010463081844322404

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [41]:
y_pred = pipeline.predict(X_test)

NotFittedError: Pipeline is not fitted yet.